In [1]:
from rubix.core.pipeline import RubixPipeline 
import os
config = {
    "pipeline":{"name": "calc_ifu"},
    
    "logger": {
        "log_level": "DEBUG",
        "log_file_path": None,
        "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    },
    "data": {
        "name": "IllustrisAPI",
        "args": {
            "api_key": os.environ.get("ILLUSTRIS_API_KEY"),
            "particle_type": ["stars"],
            "simulation": "TNG50-1",
            "snapshot": 99,
            "save_data_path": "data",
        },
        
        "load_galaxy_args": {
        "id": 14,
        "reuse": True,
        }
    },
    "simulation": {
        "name": "IllustrisTNG",
        "args": {
            "path": "data/galaxy-id-14.hdf5",
        },
    
    },
    "output_path": "output",

    "telescope":
        {"name": "MUSE"},
    "cosmology":
        {"name": "PLANCK15"},
        
    "galaxy":
        {"dist_z": 0.1}

        
}

pipe = RubixPipeline(config)

data= pipe.run()

2024-05-14 16:15:27,962 - rubix - INFO - Getting rubix data...
2024-05-14 16:15:27,963 - rubix - INFO - Rubix galaxy file already exists, skipping conversion
2024-05-14 16:15:29,905 - rubix - INFO - Data loaded with 484076 particles.
2024-05-14 16:15:29,906 - rubix - INFO - Setting up the pipeline...
2024-05-14 16:15:29,906 - rubix - DEBUG - Pipeline Configuration: {'Transformers': {'rotate_galaxy': {'name': 'rotate_galaxy', 'depends_on': None, 'args': [], 'kwargs': {'type': 'face-on'}}, 'spaxel_assignment': {'name': 'spaxel_assignment', 'depends_on': 'rotate_galaxy', 'args': [], 'kwargs': {}}}}
/home/ufuk/Dokumente/Research/Masterarbeit/rubix/rubix/telescope/factory.py:20: UserWarning: No telescope config provided, using default stored in /home/ufuk/Dokumente/Research/Masterarbeit/rubix/rubix/telescope/telescopes.yaml
  warnings.warn(
2024-05-14 16:15:30,317 - rubix - INFO - Assembling the pipeline...
2024-05-14 16:15:30,318 - rubix - INFO - Compiling the expressions...
2024-05-14 16:

rotating galaxy:  face-on


2024-05-14 16:15:30,544 - rubix - INFO - Pipeline run completed in 0.64 seconds.


In [7]:
print(data.keys())

dict_keys(['age', 'coords', 'mass', 'metallicity', 'n_particles', 'pixel_assignment', 'spatial_bin_edges', 'velocities'])


In [8]:
mass = data["mass"]
metallicity = data["metallicity"]
age = data["age"]

